In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0,'../')

import requests
from tqdm.notebook import tqdm

In [3]:
import pymongo
import pprint
import json
from src.mongo_utils import collection
from src.mlflow_setup import mlflow


api_endpoint = f"http://127.0.0.1:8000/mixtral/"

In [6]:
from src.mongo_utils import get_database_entry, create_set


with open('../test_set1.json') as file:
    test_set_data = json.load(file)
test_set = [a for subl in test_set_data.values() for a in subl]

In [7]:
dataset = []
for data in test_set:
    dataset += create_set(get_database_entry(data))

In [4]:
with open('../test_set1_postprocessed.json', 'r') as file:
    dataset = json.load(file)

In [7]:
import random
random.seed(42)
random.shuffle(dataset)

In [5]:
import pandas as pd

eval_data = {
    "inputs": [],
    "ground_truth": [],
    "prompt_data": []
}
for title, context_word, context_sentence, gt in dataset:
    test = {}
    test['title'] = title
    test['context_word'] = context_word
    test['context_sentence'] = context_sentence.replace("'", "")
    eval_data['inputs'] += [f"{context_word} --- {context_sentence}"]
    eval_data['prompt_data'] += [test]
    eval_data['ground_truth'] += [gt]
eval_data = pd.DataFrame(eval_data)
eval_data

,inputs,ground_truth,prompt_data
0,Liebe --- Die ''Liebe'' überwindet alle Grenzen.,inniges Gefühl der Zuneigung für jemanden oder...,"{'title': 'Liebe', 'context_word': 'Liebe', 'c..."
1,Liebe --- „Die ''Liebe'' ist in der Literatur ...,inniges Gefühl der Zuneigung für jemanden oder...,"{'title': 'Liebe', 'context_word': 'Liebe', 'c..."
2,Liebe --- „Im Mittelpunkt stand die ''Liebe''.“,inniges Gefühl der Zuneigung für jemanden oder...,"{'title': 'Liebe', 'context_word': 'Liebe', 'c..."
3,"Liebe --- „Die Gefühle, die sie durchströmen u...",inniges Gefühl der Zuneigung für jemanden oder...,"{'title': 'Liebe', 'context_word': 'Liebe', 'c..."
4,"Liebe --- „Erst zu Beginn der Fastenzeit, nach...",inniges Gefühl der Zuneigung für jemanden oder...,"{'title': 'Liebe', 'context_word': 'Liebe', 'c..."
...,...,...,...
1102,"Acht --- „Jetzt verhängt, wie Dr. Robert mitte...",Ausschluss aus einer Gemeinschaft; Verurteilun...,"{'title': 'Acht', 'context_word': 'Acht', 'con..."
1103,Acht --- „Seither tickt im ‚Runden Eck‘ eine Z...,Ausschluss aus einer Gemeinschaft; Verurteilun...,"{'title': 'Acht', 'context_word': 'Acht', 'con..."
1104,"Acht --- „Der Kleine widerſtrebte, ſteif und u...","Aufmerksamkeit, Beachtung; Fürsorge","{'title': 'Acht', 'context_word': 'Acht', 'con..."
1105,Acht --- „Worüber ist das Buch: sagt Marie end...,"Aufmerksamkeit, Beachtung; Fürsorge","{'title': 'Acht', 'context_word': 'Acht', 'con..."


In [6]:
import ipywidgets as widgets
out = widgets.Output(layout={'border': '1px solid black'})

In [7]:
def eval_network(data, system_prompt, question_prompt, example, out=None):
    answers = []
    for index, row in tqdm(data.iterrows(), total=len(data)):
        context_sentence = row['prompt_data']['context_sentence']
        context_word = row['prompt_data']['context_word']
        body = {
          "system_prompts": system_prompt,
          "dialog": [
              *example,
            {
              "role": "user",
              "content": question_prompt % (context_sentence, context_word)
            }
          ]
        }
        response = ""
        with requests.post(api_endpoint, json=body, stream=True) as r:
            for chunk in r.iter_content(1024):
                response += chunk.decode()
        answers.append(response)
        if out:
            with out:
                out.clear_output()
                print(f"{context_word}\n{context_sentence}\n\n{response}")

    return answers

In [8]:
out

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [ ]:
with mlflow.start_run():
    system_prompt = [
        "Du bist ein Definitionsgenerator. "
        "Du antwortest in Deutsch. "
        "Du bekommst einen Beispielsatz und ein Wort aus dem Satz. "
        "Du antwortest mit der Definition des Wortes im Kontext des Beispiels. "
        "Du antwortest nur mit der Definition. "
        "Du zeigst den Bezug auf den Kontext. "
        "Du antwortest in maximal 5 Worten. "
      ]
    question_prompt = "%s Was ist die Definition von %s?"
    example_prompt = [{
          "role": "user",
          "content": question_prompt % ("Liebe", "Die Liebe überwindet alle Grenzen")
        },
        {
          "role": "assistant",
          "content": "inniges Gefühl der Zuneigung für jemanden oder für etwas"
        }]

#    example_prompt = []
    mlflow.log_param("system_prompt", system_prompt)
    mlflow.log_param("question_prompt", question_prompt)
    mlflow.log_param("example_prompt", example_prompt)

    results = mlflow.evaluate(
        lambda x: eval_network(x, system_prompt, question_prompt, example_prompt, out),
        eval_data,
        model_type="text",
        targets="ground_truth",
        extra_metrics=[mlflow.metrics.rougeL()]#, bert_f1_score_metric]
    )    